# Top 5 Queries

# Top 1 - Chapter 7 Exercise 1
- Write a query against the dbo.Orders table that computes for each  customer order, both a rank and a dense rank, partitioned by custid, ordered by qty 
- Tables Involved: Sales.Order(Substituted with the new table created in the chapter)

Desired output:
custid |orderid    | qty         |rnk                  |drnk
------ |----------- |-----------| -------------------- |--------------------
A      |30001   |    10      |    1                    |1
A  |    40005   |    10      |    1                    |1
A   |   10001   |    12      |    3                    |2
A   |   40001   |    40      |    4                    |3
B    |  20001   |    12      |    1                    |1
B     | 30003   |    15      |    2                    |2
B      |10005   |    20      |    3                    |3
C     | 10006   |    14      |    1                    |1
C      |20002   |    20      |    2                    |2
C      |30004   |    22      |    3                    |3
D      |30007   |    30      |    1                    |1

In [ ]:
use Northwinds2022TSQLV7
select custid, orderid, qty, 
rank() over (partition by custid order by qty) as rnk, 
DENSE_RANK() over (partition by custid order by qty) as drnk 
from dbo.Orders

### Explanation:

The proposition is asking us to use the dbo.orders table that we created earlier in the chapter. I placed the new table inside the northwinds database to have consistency. I select the custid, orderid, and qty from that table. The proposition outlines the necesities for rank and denserank using the partition by clause on the custid and ordered by the qty for both columns. The rank represents how many different orders have a larger quantity while the dense rank column represents the number of orders with a distinct value larger than current row's qty value. For both columns they add one becuase it shows the rank of each row starting from 1.

# Top 2 - Chapter 7 Exercise 2

- The following query against the Sales.OrderValues view returns distinct values and their associated row numbers

```
SELECT val, ROW_NUMBER() OVER(ORDER BY val) AS rownum
FROM Sales.OrderValues
GROUP BY val;
```

- Can you think of an alternative way to achieve the same task?

Tables involved: Sales.OrderValues view

\-- Desired output:

| val | rownum |
| --- | --- |
| 12.50 | 1 |
| 18.40 | 2 |
| 23.80 | 3 |
| 28.00 | 4 |
| 30.00 | 5 |
| 33.75 | 6 |
| 36.00 | 7 |
| 40.00 | 8 |
| 45.00 | 9 |
| 48.00 | 10 |
| ... |  |
| 12615.05 | 793 |
| 15810.00 | 794 |
| 16387.50 | 795 |

(795 row(s) affected)

In [ ]:
use TSQLV6
;WITH temp AS
(SELECT distinct val FROM Sales.OrderValues)
SELECT val, ROW_NUMBER() OVER(ORDER BY val) AS rownum
FROM temp;

### Explanation:

The proposition is asking us to find an alternative way to achieve the distinct value and the row number they are in the table output. The example query uses a group by to get the distinct values so the alternative way I used was using the distinct operator to get the unique values in the val column of the view. and putting that into a cte so that we can represent the val columns as just the unique values. I select the vals from the cte and the row number on the over clause gets the row number for each value to get the desired output in a different way.

# Top 3 - Chapter 7 Q1 - Window functions, Described

- Write a query that calculates the running total value for employee and month
- Tables Involved: Sales.EmpOrders

| empid | ordermonth | val | runval |
| --- | --- | --- | --- |
| 1 | 2014-07-01 | 1614.88 | 1614.88 |
| 1 | 2014-08-01 | 5555.90 | 7170.78 |
| 1 | 2014-09-01 | 6651.00 | 13821.78 |
| 1 | 2014-10-01 | 3933.18 | 17754.96 |
| 1 | 2014-11-01 | 9562.65 | 27317.61 |
| ... |  |  |  |
| 2 | 2014-07-01 | 1176.00 | 1176.00 |
| 2 | 2014-08-01 | 1814.00 | 2990.00 |
| 2 | 2014-09-01 | 2950.80 | 5940.80 |
| 2 | 2014-10-01 | 5164.00 | 11104.80 |
| 2 | 2014-11-01 | 4614.58 | 15719.38 |
| ... |  |  |  |

(192 row(s) affected)

In [ ]:
use TSQLV6
SELECT empid, ordermonth, val,
  SUM(val) OVER(PARTITION BY empid
                ORDER BY ordermonth
                ROWS BETWEEN UNBOUNDED PRECEDING
                         AND CURRENT ROW) AS runval
FROM Sales.EmpOrders;

### Explanation:

The proposition is asking us to calculate a running total value from the view Sales.EmpOrders based on each employee and month. We select the empid and ordermonth to get the employee and the months. The running total is from a over clause with the sum of the val column partition by the employee's id that is ordered by the month. The running total is used by that with the val of the previous row and the current row to consisntly add to the sum one row at a time.

# Top 4 - Chapter 7 Q7 - Offset Window Functions, Lag and Lead

- Write a query that returns order val of current, previous, and next row from the OrderValues view
- Tables Involved: Sales.OrderValues

| custid | orderid | val | prevval | nextval |
| --- | --- | --- | --- | --- |
| 1 | 10643 | 814.50 | NULL | 878.00 |
| 1 | 10692 | 878.00 | 814.50 | 330.00 |
| 1 | 10702 | 330.00 | 878.00 | 845.80 |
| 1 | 10835 | 845.80 | 330.00 | 471.20 |
| 1 | 10952 | 471.20 | 845.80 | 933.50 |
| 1 | 11011 | 933.50 | 471.20 | NULL |
| 2 | 10308 | 88.80 | NULL | 479.75 |
| 2 | 10625 | 479.75 | 88.80 | 320.00 |
| 2 | 10759 | 320.00 | 479.75 | 514.40 |
| 2 | 10926 | 514.40 | 320.00 | NULL |
| 3 | 10365 | 403.20 | NULL | 749.06 |
| 3 | 10507 | 749.06 | 403.20 | 1940.85 |
| 3 | 10535 | 1940.85 | 749.06 | 2082.00 |
| 3 | 10573 | 2082.00 | 1940.85 | 813.37 |
| 3 | 10677 | 813.37 | 2082.00 | 375.50 |
| 3 | 10682 | 375.50 | 813.37 | 660.00 |
| 3 | 10856 | 660.00 | 375.50 | NULL |
| ... |  |  |  |  |

(830 row(s) affected)

In [ ]:
use TSQLV6
SELECT custid, orderid, val,
  LAG(val)  OVER(PARTITION BY custid
                 ORDER BY orderdate, orderid) AS prevval,
  LEAD(val) OVER(PARTITION BY custid
                 ORDER BY orderdate, orderid) AS nextval
FROM Sales.OrderValues
ORDER BY custid, orderdate, orderid;

### Explanation:

The proposition is asking us to use the view, Sales.OrderValues to get the order val of the current, previous, and next val order. I start by selecting the custid, orderid, and val for the order and the next column finds the previous val number. Using the lag offset function taking the val column and adding a over clause to a partition by on the custid and ordered by the date and id of the orders. Repeat the syntax excpet switch lag with lead to get the next val and it creates the desired output.

# Top 5 - Chapter 7 Q8- Offset Window Functions, First\_Value and Last\_Value

- Write a query that returns the value of the current, first, and last orders for each customer
- Tables Involved: Sales.OrderValues

| custid | orderid | val | firstval | lastval |
| --- | --- | --- | --- | --- |
| 1 | 10643 | 814.50 | 814.50 | 933.50 |
| 1 | 10692 | 878.00 | 814.50 | 933.50 |
| 1 | 10702 | 330.00 | 814.50 | 933.50 |
| 1 | 10835 | 845.80 | 814.50 | 933.50 |
| 1 | 10952 | 471.20 | 814.50 | 933.50 |
| 1 | 11011 | 933.50 | 814.50 | 933.50 |
| 2 | 10308 | 88.80 | 88.80 | 514.40 |
| 2 | 10625 | 479.75 | 88.80 | 514.40 |
| 2 | 10759 | 320.00 | 88.80 | 514.40 |
| 2 | 10926 | 514.40 | 88.80 | 514.40 |
| 3 | 10365 | 403.20 | 403.20 | 660.00 |
| 3 | 10507 | 749.06 | 403.20 | 660.00 |
| 3 | 10535 | 1940.85 | 403.20 | 660.00 |
| 3 | 10573 | 2082.00 | 403.20 | 660.00 |
| 3 | 10677 | 813.37 | 403.20 | 660.00 |
| 3 | 10682 | 375.50 | 403.20 | 660.00 |
| 3 | 10856 | 660.00 | 403.20 | 660.00 |
| ... |  |  |  |  |

(830 row(s) affected)

In [ ]:
use TSQLV6
SELECT custid, orderid, val,
  FIRST_VALUE(val) OVER(PARTITION BY custid
                        ORDER BY orderdate, orderid
                        ROWS BETWEEN UNBOUNDED PRECEDING
                                 AND CURRENT ROW) AS firstval,
  LAST_VALUE(val)  OVER(PARTITION BY custid
                        ORDER BY orderdate, orderid
                        ROWS BETWEEN CURRENT ROW
                                 AND UNBOUNDED FOLLOWING) AS lastval
FROM Sales.OrderValues
ORDER BY custid, orderdate, orderid;

### Explanation:

The proposition is asking us to use the view, Sales.OrderValues to get the order val of the current, first, and last val order for each customer. I start by selecting the custid, orderid, and val for the order and the next column finds the value of the first order for that customer. Using the First Value offset function taking the val column and adding a over clause to a partition by on the custid and ordered by the date and id of the orders. Then we add a Rows Between clause to filter the rows from a unbounded point to the current row. Repeat the syntax excpet switch first with last to get the last value with the rows between looking from the current row and no upper bound and it creates the desired output.

# Chapter 7 Q2 - Ranking Window functions

- Write a query that ranks the values using all the ranking functions
- Tables Involved: Sales.OrderValues

In [ ]:
Use TSQLV6
SELECT orderid, custid, val,
  ROW_NUMBER() OVER(ORDER BY val) AS rownum,
  RANK()       OVER(ORDER BY val) AS rank,
  DENSE_RANK() OVER(ORDER BY val) AS dense_rank,
  NTILE(10)    OVER(ORDER BY val) AS ntile
FROM Sales.OrderValues
ORDER BY val;

### Explanation:

We need to use all the ranking functions available to rank the values from the view Sales.OrderValues, so we just use the ranking functions row\_number, rank, dense\_rank, and ntile with all having the same over clause that is ordered by the value. The only difference for this specific case is the ntile function that breaks up the rows into 10 groups and evenly assigning the values so that there is about the same number of rows with a rank of each number from 1-10.

# Chapter 7 Q3 - Ranking Window functions

- Write a query that numbers each order for each customer
- Tables Involved: Sales.OrderValues

In [ ]:
use TSQLV6
SELECT orderid, custid, val,
  ROW_NUMBER() OVER(PARTITION BY custid
                    ORDER BY val) AS rownum
FROM Sales.OrderValues
ORDER BY custid, val;

### Explanation:

We get the regular information of the order with the cutsomer's id, the order's id, and the value of the order, and to number each order for the customer we use the row-number function with a over clause that partitions the customer id and orders by the val to assign a rank to each order from lowest to highest.

# Chapter 7 Q4 & 5 - Ranking Window functions

- Write a query that ranks each unique value number

```
SELECT DISTINCT val, ROW_NUMBER() OVER(ORDER BY val) AS rownum
FROM Sales.OrderValues;
```

- Why does this attempt not produce the correct output and what is way achieve the desired output
- Tables Involved: Sales.OrderValues

In [ ]:
use TSQLV6
SELECT val, ROW_NUMBER() OVER(ORDER BY val) AS rownum
FROM Sales.OrderValues
GROUP BY val;

### Explanation:

The first attempt at answering the proposition doesn't work becuase the row\_number clause is proccssed before the distinct clause so it already has ranked each value making it unique and the distinct will not see any duplicates. Fixing this we instead use group by in place of distinct so that we can have one single use of each value and then numbering the rows to get no duplicate value numbers.